In [1]:
import os
import sys
import pandas as pd
import numpy as np
#sys.path.append(os.path.join('..', 'MarketingMixModel'))
#from MarketingMixModel import MarketingMixModel

In [2]:
for dataset_size in [1000, 100000]:
    for sderr in [1, 50]:
        print(f'Running set size {dataset_size} and stderr {sderr}')
        # random bits of food from 1 to 10000 each period
        data = np.random.randint(1, 10000, size=dataset_size)
        df = pd.DataFrame(data, columns=['food'])
        df['poison'] = 0
        df= df.reset_index()
        # two periods where miniscule amount of poison are consumed.
        # but even small amounts of poison are devasating although this is not done much
        df.loc[df.index==3, 'poison'] = 1
        df.loc[df.index==8, 'poison'] = 2

        # specify the underlying VAR equation which we intend to reverse engineer

        # matrix math formula
        # health = health t-1 + health_const * food t-1 * food_const * poison t-1 * poison_const + random variable

        health_const = 0.999999
        food_const = 0.001
        poison_const = -500

        df['health'] = 10000 # base level
        for i in range(1, dataset_size): # level we intend to fill in
            df.loc[df.index==i, 'health'] = df.at[i-1, 'food'] *food_const  + df.at[(i-1), 'health']*health_const + df.at[(i-1), 'poison'] * poison_const + np.random.normal(scale=sderr, size=1)

        # in this case, we think that food and poison do not really depend on health, although they are specified as endogenous!

        df.to_csv(os.path.join('data', f'poison_{dataset_size}_{sderr}err.csv'))

Running set size 1000 and stderr 1
Running set size 1000 and stderr 50
Running set size 100000 and stderr 1
Running set size 100000 and stderr 50
